# Manipuler des fichiers

## Ouvrir un fichier

Comme pour un livre, un fichier doit s’ouvrir et se fermer. Les deux opérations se réalisent en une seule instruction, grâce au mot-clé `with`, accompagné de la fonction `open()`, qui indique au langage qu’il doit travailler avec un objet, lui laissant le soin de fermer le flux vers le fichier à la fin du bloc d’instructions.

La fonction `open()` accepte deux arguments. Le premier est le chemin vers le fichier à ouvrir. S’il se trouve dans le répertoire courant, son nom seul est requis. Le second argument est le mode d’ouverture.

Il existe trois modes d'ouverture de fichier :
- **Mode lecture seule :** aucune écriture autorisée dans le fichier ;
- **Mode écriture :** écriture à partir du début du fichier, ce qui signifie qu’un fichier du même nom pré-existant sera systématiquement remplacé ;
- **Mode ajout :** écriture à la fin du fichier.

In [ ]:
with open("./data/salammbo.txt", "r") as src:
    print(src)

Ici, le mode d’ouverture est *read* `r`. On aurait pu en faire l’économie, car le mode en lecture seule est le mode par défaut. Les autres modes possibles sont `w` pour *write* et `a` pour *append*.

## Lire un fichier

Il existe plusieurs méthodes pour afficher les informations contenues dans un fichier. Avant d’opter pour l’une ou l’autre, il convient de se poser la question de la finalité du programme :
- La méthode `read()` pour afficher les données au format brut, sous forme de chaîne de caractères.
- La méthode `readline()` pour afficher la ligne courante au format brut, augmentée d’un retour à la ligne, sous forme de chaîne de caractères.
- La méthode `readlines()` pour afficher les données au format brut, sous forme de liste.

### Méthode `read()`

Elle prend un argument numérique facultatif, qui représente le nombre de caractères que vous souhaitez afficher à partir de la position courante du curseur. Par défaut, elle renvoie la totalité des caractères du fichier.

In [ ]:
# 'src' is a file descriptor
with open("./data/salammbo.txt", "r") as src:
    # Reads the first 31 characters
    print(src.read(31))
    # Reads the next 69 characters
    print(src.read(69))

### Méthode `readline()`

Elle affiche la ligne courante du fichier, dans la limite du nombre de caractères défini en paramètre par un argument numérique. Chaque appel à la méthode génère une nouvelle ligne `\n`.

In [ ]:
with open("./data/salammbo.txt", "r") as src:
    # The first and the second lines
    print(src.readline())
    print(src.readline())
    # The first 40 characters of the third line
    print(src.readline(40))
    # The remaining characters of the third line
    print(src.readline())

Grâce à cette méthode, on peut concevoir un algorithme très simple pour parcourir un fichier ligne par ligne :
```txt
Ouverture d'une ressource :
    Tant qu'il reste des lignes à afficher :
        Afficher la ligne courante
```

In [ ]:
with open("./data/salammbo.txt") as src:
    while src.readline():
        print(src.readline())

### Méthode `readlines()`

Plutôt que de retourner une chaîne de caractères, cette méthode retourne une liste des lignes du fichier. Elle opère la séparation aussitôt qu’elle rencontre un caractère de nouvelle ligne `\n`.

In [ ]:
with open("./data/salammbo.txt") as src:
    print(src.readlines()[:4])

Comme le caractère `\n` est préservé à la fin de chaque ligne, il faut utiliser la méthode `strip()` pour le retirer :

In [ ]:
with open("./data/salammbo.txt") as src:
    print(src.readlines()[2].strip())

## Écrire dans un fichier

Le pré-requis pour écrire dans un fichier est d’abord de l’ouvrir en mode écriture (`w` ou `a`). S’il n’existe pas déjà de fichier avec le même nom, les deux modes en génèrent un et insèrent le contenu à l’intérieur. Si au contraire un fichier du même nom est trouvé, le mode `w` remplace complètement le contenu quand le mode `a` ajoute le contenu à écrire au contenu existant.

La méthode `write` sur le descripteur de fichier se charge d’écrire le contenu, forcément de type `String`.

Créer un nouveau fichier :

In [ ]:
with open("./data/the-dark-hole.txt", "w") as file:
    file.write("The Dark Hole")

Remplacer le contenu du fichier :

In [ ]:
with open("./data/the-dark-hole.txt", "w") as file:
    file.write("No escape from a Dark Hole")

Ajouter du contenu à un fichier existant :

In [ ]:
with open("./data/the-dark-hole.txt", "a") as file:
    file.write("\n")
    file.write("The whole universe will perish inside a dark hole.")

## Techniques courantes

### L’itérateur de fichier

Si l’objectif est d’itérer sur un fichier, il est inutile de récupérer au préalable la liste des lignes avec par exemple la méthode `readlines()`. L’objet qui pointe vers le fichier dispose en effet de son propre itérateur :

In [ ]:
with open("./data/salammbo.txt") as src:
    for line in src:
        print(line)

### Lecture séquentielle

Avec un fichier aussi léger (en poids) que le *Salammbô* de Flaubert, la lecture en un seul bloc ne présente aucun enjeu. En revanche, si l’on effectue un traitement sur de vastes corpus de données, l’accès aux ressources peut à tout moment souffrir des aléas de l’informatique :
- une coupure électrique ;
- la diminution des ressources allouées par le système aux scripts trop gourmands ;
- un plantage du serveur ;
- un arrêt brutal du script par un administrateur ;
- …

Bref, il peut arriver tellement de choses à tout moment que l’on va préférer lire un fichier de manière séquentielle plutôt que d’un seul tenant. C’est la raison d’être des itérateurs mais, si l’on souhaite plutôt utiliser la méthode `read()` pour parcourir un fichier, on va décider du nombre de caractères à traiter à la fois :

In [ ]:
with open("./data/salammbo.txt") as src:
    while True:
        txt = src.read(50)
        if not txt: break
        else: print(txt)

### Créer un fichier plat

Les fichiers plats (ou bases de données orientées texte) sont des fichiers qui entreposent des données sous un format uniforme sans structure particulière pour indiquer leur modèle relationnel. Chaque enregistrement occupe une ligne du fichier et au sein d’un même enregistrement, les données sont séparées les unes des autres par un délimiteur.

```txt
Le;DET;le
petit;ADJ;petit
chat;NC;chat
est;V;être
mort;ADJ;mort
.;PONCT;.
```

Sans surprise, le format CSV (*Comma-Separated Values*) est un exemple de fichier plat. Formellement, rien ne le distingue d’un fichier TXT, et la fonction native `open()` le traite de la même manière, en lecture comme en écriture :

In [ ]:
with open('./data/covid.csv') as src:
    for line in src.readlines()[:5]:
        print(line)